In [1]:
import numpy as np
import SimpleITK as sitk
import pyvista as pv
from pyvista import examples
from numba import njit, prange
from skimage import measure
import scipy.io as sio
from pyvistaqt import BackgroundPlotter

In [2]:
import pdb
def lines_from_points(points):
    """Given an array of points, make a line set"""
    poly = pv.PolyData()
    poly.points = points
    cells = np.full((len(points) - 1, 3), 2, dtype=np.int_)
    cells[:, 1] = np.arange(0, len(points) - 1, dtype=np.int_)
    cells[:, 2] = np.arange(1, len(points), dtype=np.int_)
    poly.lines = cells
    return poly


def create_tubes(start_point, end_point, query, streamline_width=0.0001):
    tubes = []

    norm_query = query / np.max(np.abs(query))

    # Create tubes
    for element in range(len(query)):
        points = np.vstack([start_point[element], end_point[element]])
        #pdb.set_trace()
        line = lines_from_points(points)
        line["query"] = np.array([query[element]])

        tubes.append(line.tube(radius=streamline_width * norm_query[element]))

    return tubes

def create_spheres(branch_points,rad=1,res=30):
    sphere = pv.Sphere(radius=rad)#theta_resolution=res,phi_resolution=res)

    spheres = []
    for coords in branch_points:

        # create a "point cloud" containing the centers of every cube
        mesh = pv.PolyData(coords)

        # use a glyph filter to create a cube at each point in polydata
        glyphs = mesh.glyph(geom=sphere,scale=True)
        spheres.append(glyphs)
    return spheres

In [4]:
mat_contents = sio.loadmat('/Users/noahschweitzer/Library/CloudStorage/OneDrive-UniversityofPittsburgh/TOF_humans/For_steve/for_pyvista_visualizing.mat')
start_point = mat_contents['start_point']
end_point = mat_contents['end_point']
query_data = mat_contents['query_data']
branch_points = mat_contents['branchpoints']

KeyError: 'branchpoints'

In [ ]:
tubes = create_tubes(start_point, end_point, query_data , streamline_width=0.5)

In [ ]:
spheres = create_spheres(branch_points)

In [ ]:
mesh2 = pv.MultiBlock(tubes).combine().extract_surface().clean() # see note regarding merge vs boolean add

In [ ]:
mesh = pv.MultiBlock(spheres).combine().extract_surface().clean() # see note regarding merge vs boolean add

In [ ]:
pv.set_plot_theme("document")  ## white background

plotter = BackgroundPlotter()
plotter.add_mesh(mesh,color='red') #
plotter.add_mesh(mesh2, scalars="query", cmap="viridis")
plotter.show_axes()

In [ ]:
tubes[0]

In [ ]:
spheres[0]